In [ ]:
# Unita: GDT

## Semplice classificazione basata su Logistic Regression e Tensorflow.

In [ ]:
# in seguito ci servono
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range


In [ ]:
# partiamo dall'output dell'unità 04
# folder dove e' posizionato il file pickle
pickle_file = "../datasets/notMNIST.pickle"

In [ ]:
# carico i dati
with open(pickle_file, 'rb') as f:
  saved = pickle.load(f)
  train_dataset = saved['train_dataset']
  train_labels = saved['train_labels']
  valid_dataset = saved['valid_dataset']
  valid_labels = saved['valid_labels']
  test_dataset = saved['test_dataset']
  test_labels = saved['test_labels']
  del saved  # garbage collector per liberare memoria
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
print(train_dataset[0])
print(train_labels[0].shape)

In [ ]:
# Riportiamo i dati nel formato adatto al processamento: matrix 1-dim + vettore 1-hot encoding
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  # -1 indica che la dimensione iniziale rimane invariata
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  # aggiungo una dimensione a labels
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


In [ ]:
print(train_dataset[0])
print(train_labels[0].shape)

In [ ]:
## PROBLEMA #1
# Implementare una logistic regression multinomiale con discesa del gradiente 
# con Tensorflow (TF) come classificatore per notMNIST.

In [ ]:
# La matrice dei pensi W viene spesso inizializzata con una variabile casuale con distribuzione normale,
# dove i valori maggiori di 2 x std_dev sono rimossi.
# weights = tf.Variable( tf.truncated_normal(...))

# Questo permette di ignorare valori troppo grandi o piccoli che possono influenzare negativamente l'apprendimento.
import matplotlib.pyplot as plt

%matplotlib inline  

n = 500000
A = tf.truncated_normal((n,))
B = tf.random_normal((n,))
with tf.Session() as sess:
    a, b = sess.run([A, B])

plt.hist(a, 100, (-4.2, 4.2));

In [ ]:
plt.hist(b, 100, (-4.2, 4.2));

In [ ]:
# Prendiamo un subset per limitare il tempo per l'addetramento, diminuiscilo se occorre
train_subset = 10000

# Su Tensorflow ogni elemento - input, variabili ed elaborazioni - è descritto mediante un grafo, o dataflow graph.
# Gli oggetti tf.Operation rappresentano unità di computazione;
# Gli oggetti tf.Tensor rappresentano unità di dati (tensori) che sono usati come input e output per gli oggetti Operation.

# In TF un grafo tf.Graph contiene due tipi di informazione:
# La struttura: nodi e archi che rappresentano le operazioni 
# Le collections: insiemi di metadati (inseriti con tf.add_to_collection) nella forma <chiave,lista di objects); si può ispezionare con tf.get_collection.

# TF usa questa struttura per salvare variabili e altre informazioni del grafo.

# Un oggetto Graph di default è sempre prensente e accedibile chiamando tf.get_default_graph. 

# Un approccio alternativo per usare i grafo di Tensorflow consiste nel context manager tf.Graph.as_default, 
# che sostituisce il grafo di default per tutta l'esistenza del contesto in esame.
graph = tf.Graph()

# Costruisco un grafo di computazione con Tensorflow
with graph.as_default():

  # Creo tensori costanti per i seguenti set: trainig, test e validation
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  # per assegnare un nome alla variabile possiamo usare il secondo parametro, e.g., tf.constant(0, name="c") 
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Le variabili mantengono lo stato durante le elaborazioni. Sono anch'esse tensori.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  # Il vettore di bias b è inizializzato a 0.
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Calcolo Wx + b
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # La funzione softmax_cross_entropy_with_logits valuta la funzione di loss
  # per mezzo della cross-entropy loss con l'output corretto (tf_train_labels)
  # Mentre reduce_mean valuta semplicemente la media dei valori del tensore.
  # loss indica una operazione TF.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Instanzio un algoritmo di discesa del gradiente con learning rate = 0.5 (alfa nelle slide di richiami sulle reti neurali.)
  # La funzione minimize è composta di 2 elaborazioni: compute_gradients e apply_gradients.
  # La prima ricava i gradienti, la seconda aggiorna la matrice dei pesi di conseguenza.
  # optimizer indica una operazione TF.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  # calcolo softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), dim) per i sets: 
  # training, validation e test.
  # N.B.: i set valid e test sono usati solo per la valutazione, non c'è backprop
  # N.B.(2): ci servono per valutare l'accuratezza, l'apprendimento l'abbiamo già fatto.
  train_prediction = tf.nn.softmax(logits)
  logits = tf.matmul(tf_valid_dataset, weights) + biases
  valid_prediction = tf.nn.softmax(logits) 
  logits = tf.matmul(tf_test_dataset, weights) + biases
  test_prediction = tf.nn.softmax(logits)
        

In [27]:
# numero cicli di elaborazione
num_steps = 801

# Definisco l'accuratezza come somma del numero di predizioni corrette 
# normalizzato sul numero di predizioni totali.
# La uso per fare statistiche durante il funzionamento.
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# TF usa tf.Session per rappresentare una connesione tra il programa e il runtime C++.
# Serve per creare un ambiente in cui lanciare le operazioni definite nel grafo.
# Poichè la classe alloca risorse fisiche, solitamente si usa come context manager (dentro un blocco with),
# che le libera automaticamente al termine del blocco, cioè lancia session.close() al termine della esecuzione.
with tf.Session(graph=graph) as session:
  # Istanzia e lancia una operazione per l'inizializzazione delle variabili globali del grafo
  # cioè: weights e biases. Va eseguita solo una volta.
  tf.global_variables_initializer().run()
  print('Initialized')
      
  for step in range(num_steps):
    # Eseguo le operazioni nel grafo.
    # Le operazioni e i tensori da valutare sono definiti nel primo parametro, un NumPy array.
    # La lista indica le foglie grafo.
    # Il valore di ritorno ha lo stesso tipo dell'input, cioè un array, 
    # dove le foglie sono sostituite con il corrispondente valore calcolato da TF.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    # ogni tanto stampo statistiche
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Se invoco eval() su valid_prediction, sto calcolando l'operazione sui 
      # pesi e bias correnti.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  # Al termine stamo l'accuracy sul test set.
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Loss at step 400: 1.461176
Training accuracy: 77.2%
Validation accuracy: 75.4%
Loss at step 500: 1.335957
Training accuracy: 77.9%
Validation accuracy: 75.7%
Loss at step 600: 1.238967
Training accuracy: 78.5%
Validation accuracy: 76.0%
Loss at step 700: 1.160774
Training accuracy: 79.0%
Validation accuracy: 76.1%
Loss at step 800: 1.095832
Training accuracy: 79.5%
Validation accuracy: 76.4%
Test accuracy: 83.2%


In [ ]:
## PROBLEMA #2
# Prova a modificare il codice precedente impiegando un Stochastic gradient descent.
# Quanto tempo impiega ora per terminare l'elaborazione?

In [ ]:
# Durante l'elaborazone batch l'algoritmo elabora solo un sottoinsieme di dati alla volta.
# L'elaborazione è ripetuta, perciò conviene scrivere il codice 
# senza gestire la creazione dei dati direttamente.
# In TF un placeholder è una variabile che assumera i valori a tempo di esecuzione.
# Possiamo costruire il grafo delle operazioni senza il bisogno di conoscere i dati.

# Nel seguente codice creiamo una operazione (y) di moltiplicazione * 2 senza sapere i valori.
# Ora la possiamo eseguire all'interno di una sessione. Per valutarla occorre fornire (feed)
# i valori per x. 
# None significa che non poniamo vincoli sulla dimensione.
x = tf.placeholder(tf.float32, shape=None)
y = x * 2

# TF supporta tipi di variabili simili a NumPy (es. float32, float64, int32, int64)
# https://docs.scipy.org/doc/numpy-1.13.0/user/basics.types.html

with tf.Session() as session:
    result = session.run(y, feed_dict={x: [1, 2, 3]})
    print(result)

In [ ]:
# N.B. Fuori dallo scope session non possiamo stampare il valore dei tensori 
# durante l'elaborazione.

# x = tf.placeholder("float", None)
# y = x * 2
# print(x) 

# Output: "Tensor("Placeholder_11:0", dtype=float32)" 
# Stampa solo il tipo e non il valore di x.
# In alternativa usare https://www.tensorflow.org/api_docs/python/tf/InteractiveSession

In [ ]:
# Possiamo dare in input anche strutture più complesse indicando il formato dei dati con shape.
# Es. un qualsiasi numero di righe, ma il numero di colonne pari a 3
x = tf.placeholder(tf.float32, shape=[None, 3])
y = x * 2

with tf.Session() as session:
    x_data = [[1, 2, 3],
              [4, 5, 6],]
    result = session.run(y, feed_dict={x: x_data})
    print(result)

In [ ]:
# Una immagine a colori (RGB) in formato raw può avere una rappresentazione matriciale
#image = tf.placeholder("uint8", shape=[None, None, 3])

In [ ]:
## PROBLEMA #3 (da completare dopo la lezione "2017 ML-07-`")
# Usando l'help online di TF prova a creare una rete neurale con 1-hidden layer
# con attivazione RELU e 1024 nodi nascosti.

# N.B. la funzione tf.nn.relu() restituisce un tensore che calcola la RELU sul tensore di input.
# L'output ha la stessa dimensione dell'input.